<a href="https://colab.research.google.com/github/NarasimhaMurthy2003/Crypto_Currency/blob/main/Crypto_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd


In [2]:
api_key="41d14645-cb60-4ef6-8083-de73f34e882b"
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": api_key,
}
params = {
    "start": "1",
    "limit": "50",
    "convert": "USD",
    "sort": "market_cap"
}


In [3]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

In [4]:
def fetch_coinmarketcap_data():
    response = requests.get(url, headers=headers, params=params)
    if response.status_code ==200:
      data = response.json()
      crypto_data=data["data"]
      crypto_list = [
            {
                "Name": crypto["name"],
                "Symbol": crypto["symbol"],
                "Current Price (USD)": crypto["quote"]["USD"]["price"],
                "Market Cap": crypto["quote"]["USD"]["market_cap"],
                "24h Volume": crypto["quote"]["USD"]["volume_24h"],
                "24h Price Change (%)": crypto["quote"]["USD"]["percent_change_24h"],
            }
            for crypto in crypto_data
        ]
      return pd.DataFrame(crypto_list)
    else:
        print("Failed to fetch data:", response.status_code, response.text)
        return pd.DataFrame()

In [5]:
crypto_df = fetch_coinmarketcap_data()
print(crypto_df)

                                     Name  Symbol  Current Price (USD)  \
0                                 Bitcoin     BTC         97033.005491   
1                                Ethereum     ETH          3128.385874   
2                             Tether USDt    USDT             1.000546   
3                                  Solana     SOL           238.225284   
4                                     BNB     BNB           610.842717   
5                                     XRP     XRP             1.111139   
6                                Dogecoin    DOGE             0.384237   
7                                    USDC    USDC             0.999779   
8                                 Cardano     ADA             0.782131   
9                                    TRON     TRX             0.197370   
10                              Shiba Inu    SHIB             0.000024   
11                                Toncoin     TON             5.435992   
12                              Avalan

Data Analysis


In [6]:
analysis={}
top_5=crypto_df.nlargest(5, "Market Cap")
average_price=crypto_df['Current Price (USD)'].mean()
highest_change = crypto_df.loc[crypto_df["24h Price Change (%)"].idxmax()]
lowest_change = crypto_df.loc[crypto_df["24h Price Change (%)"].idxmin()]
analysis["Top 5 Cryptocurrencies by Market Cap"] = top_5.to_dict(orient="records")
analysis["Average Crypto Price"] = average_price
analysis["Crypto with Highest 24h Price Change"] = highest_change.to_dict()
analysis["Crypto with Lowest 24h Price Change"] = lowest_change.to_dict()

In [7]:
analysis['Average Crypto Price']

2053.2824446670675

In [8]:
analysis["Top 5 Cryptocurrencies by Market Cap"]

[{'Name': 'Bitcoin',
  'Symbol': 'BTC',
  'Current Price (USD)': 97033.00549074575,
  'Market Cap': 1919824988809.9312,
  '24h Volume': 89862645299.47198,
  '24h Price Change (%)': 4.94477265},
 {'Name': 'Ethereum',
  'Symbol': 'ETH',
  'Current Price (USD)': 3128.385874487285,
  'Market Cap': 376743816579.249,
  '24h Volume': 34547928322.92176,
  '24h Price Change (%)': 0.67985055},
 {'Name': 'Tether USDt',
  'Symbol': 'USDT',
  'Current Price (USD)': 1.0005456325130127,
  'Market Cap': 130222065993.25233,
  '24h Volume': 161981332302.35294,
  '24h Price Change (%)': -0.0277404},
 {'Name': 'Solana',
  'Symbol': 'SOL',
  'Current Price (USD)': 238.22528426902204,
  'Market Cap': 113062471383.44182,
  '24h Volume': 7147075508.840479,
  '24h Price Change (%)': 1.41177607},
 {'Name': 'BNB',
  'Symbol': 'BNB',
  'Current Price (USD)': 610.8427166921025,
  'Market Cap': 87967869432.99146,
  '24h Volume': 2379893989.5648108,
  '24h Price Change (%)': -0.32433341}]

In [9]:
analysis["Crypto with Highest 24h Price Change"]

{'Name': 'Bitcoin Cash',
 'Symbol': 'BCH',
 'Current Price (USD)': 525.9096124979452,
 'Market Cap': 10408356027.388708,
 '24h Volume': 1355416851.279043,
 '24h Price Change (%)': 19.89665805}

In [10]:
analysis["Crypto with Lowest 24h Price Change"]

{'Name': 'Render',
 'Symbol': 'RENDER',
 'Current Price (USD)': 7.283597344888607,
 'Market Cap': 3770650994.938214,
 '24h Volume': 1020749522.3046435,
 '24h Price Change (%)': -10.1902866}

LIVE UPDATING EXCEL

In [11]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook, load_workbook

In [12]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import requests
import time

In [13]:
def write_to_excel(df, file_name="crypto_live_data.xlsx"):
    try:
        try:
            wb = load_workbook(file_name)
            sheet = wb.active
        except FileNotFoundError:
            wb = Workbook()
            sheet = wb.active


        headers = list(df.columns)
        sheet.append(headers)


        for row in df.itertuples(index=False):
            sheet.append(row)
        row=([])
        sheet.append(row)


        wb.save(file_name)
        print(f"Data written to {file_name}")
    except Exception as e:
        print(f"Error writing to Excel: {e}")

In [14]:
import time
def live_update(interval=300, file_name="crypto_live_data.xlsx"):
    while True:
        df = fetch_coinmarketcap_data()
        if df is not None:
            write_to_excel(df, file_name)
        else:
            print("Failed to fetch data.")
        print(f"Waiting for {interval} seconds before the next update...")
        time.sleep(interval)


In [15]:
live_update()

Data written to crypto_live_data.xlsx
Waiting for 300 seconds before the next update...
Data written to crypto_live_data.xlsx
Waiting for 300 seconds before the next update...


KeyboardInterrupt: 

In [16]:
df=pd.read_excel("crypto_live_data.xlsx")


In [17]:
!pip install ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=a366789c02e0751bb31df747dae5384fec195fd353a3dcaad125c90a4a1bb352
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [18]:
df

,Name,Symbol,Current Price (USD),Market Cap,24h Volume,24h Price Change (%)
0,Bitcoin,BTC,97033.005491,1919824988809.930908,89862645299.471985,4.944773
1,Ethereum,ETH,3128.385874,376743816579.249023,34547928322.921761,0.679851
2,Tether USDt,USDT,1.000546,130222065993.252304,161981332302.352905,-0.02774
3,Solana,SOL,238.225284,113062471383.441803,7147075508.840479,1.411776
4,BNB,BNB,610.842717,87967869432.991455,2379893989.564811,-0.324333
...,...,...,...,...,...,...
96,Cosmos,ATOM,6.248531,2442764392.443284,326234518.772182,-1.436695
97,Aave,AAVE,158.08521,2368071091.917189,271008288.616736,-1.846298
98,Injective,INJ,23.594243,2332244224.099604,209037319.959466,-1.102832
99,First Digital USD,FDUSD,0.997786,2302984432.279636,9901245480.982664,-0.018048


In [19]:
from ydata_profiling import ProfileReport

In [20]:
profile=ProfileReport(df)
profile.to_file(output_file="crypto.pdf")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:379: UserWarning: Extension .pdf not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]